Igual que en el ejercicio de pair programming de ETL Transformación II, tendréis que crear una clase que nos permita cargar los datos en SQL I.

In [108]:
import pandas as pd
import mysql.connector
pd.options.display.max_columns=None
pd.options.display.float_format = '{:.2f}'.format

In [109]:
df=pd.read_csv('datos/merge_clima_ataques.csv',index_col=0)
df.dropna(axis=0, how="any", subset=None,inplace= True)


In [110]:
df.dtypes

id_paises            object
wind_950mb           object
wind_900mb           object
wind_850mb           object
wind_800mb           object
wind_750mb           object
wind_700mb           object
wind_650mb           object
wind_600mb           object
wind_550mb           object
wind_500mb           object
wind_450mb           object
wind_400mb           object
wind_350mb           object
wind_300mb           object
wind_250mb           object
wind_200mb           object
rh950mb              object
rh900mb              object
rh850mb              object
rh800mb              object
rh750mb              object
rh700mb              object
rh650mb              object
rh600mb              object
rh550mb              object
rh500mb              object
rh450mb              object
rh400mb              object
rh350mb              object
rh300mb              object
rh250mb              object
rh200mb              object
timepoint            object
cloudcover           object
highcloud           

In [119]:
class Cargar_crear:
    
    def __init__(self,bbdd):
        
        self.bbdd = bbdd
    
    
    def crear_bbdd(self):

        mydb = mysql.connector.connect(host="localhost",
                                        user="root", 
                                        password="AlumnaAdalab")
        
        print("Conexión realizada con éxito")
        
        mycursor = mydb.cursor()

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.bbdd};")
            print(mycursor)
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)

    def crear_insertar_tabla(self,query):
    
        mydb = mysql.connector.connect(host="127.0.0.1",
                                    user="root",
                                    password="AlumnaAdalab", 
                                    database=f"{self.bbdd}") 
        mycursor = mydb.cursor()
        
        try:
            mycursor.execute(query)
            mydb.commit()
        
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
            

    def sacar_id_pais(self, pais):
    
        mydb = mysql.connector.connect(host="127.0.0.1",
                                    user="root",
                                    password="AlumnaAdalab", 
                                    database=f"{self.bbdd}") 
        mycursor = mydb.cursor()
        
        try:
            sacar_id = f"SELECT id FROM ataques WHERE pais = '{pais}'"
            mycursor.execute(sacar_id)
            id_paises = mycursor.fetchall()[0][0]
            return id_paises
        
        except: 
            return "No tenemos ese país en la BBDD y por lo tanto no te podemos sacar su ID"
        
    def insertar_datos_ataques(self,df):
        
        for indice, fila in df.iterrows(): # iteramos por el dataframe.

        # definimos nuestra query, igual que si lo hicieramos en workbench. ⚠️ Como estamos definiendo nuestra query en varias líneas usamos las triples comillas
        # lo valores que introduciremos serán los del dataframe que estamos iterando, por lo que usaremos los formats de los strings. 

            query_ataque = f"""
                INSERT INTO ataques (id_paises,year,age,latitud_x,longitud_x,fatal_N)
                VALUES ( "{fila["id_paises"]}", "{fila['year']}", "{fila['age']},"{fila['latitud_x']}","{fila['longitud_x']}","{fila['fatal_N']}");
                    """
        # una vez definida la query llamamos a la función que nos inserta los datos. 
            self.crear_insertar_tabla(query_ataque)

        def insertar_datos_clima(self,df):
            
            for indice, fila in df.iterrows():

                id_paises=self.sacar_id_pais(fila['id_paises'])
        
                query_clima = f"""
                            INSERT INTO clima (wind_950mb,wind_900mb,wind_850mb,wind_800mb,wind_750mb,wind_700mb,wind_650mb,\
                                wind_600mb,wind_550mb,wind_500mb,wind_450mb,wind_400mb,wind_350mb,wind_300mb,wind_250mb,wind_200mb,\
                                rh950mb,rh900mb,rh850mb,rh800mb,rh750mb,rh700mb,rh650mb,rh600mb,rh550mb, rh500mb,rh450mb,rh400mb,rh350mb,\
                                rh300mb,rh250mb,rh200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,\
                                prec_amount,snow_depth,wind10M.speed,latitud_y,longitud_y,id) 
                            VALUES ("{fila['wind_950mb']}","{fila['wind_900mb']}","{fila['wind_850mb']}","{fila['wind_800mb']}","{fila['wind_750mb']}", 
                            "{fila['wind_700mb']}","{fila['wind_650mb']}","{fila['wind_600mb']}","{fila['wind_550mb']}","{fila['wind_500mb']}","{fila["wind_450mb"]}", 
                            "{fila['wind_400mb']}","{fila['wind_350mb']}","{fila['wind_300mb']}","{fila['wind_250mb']}","{fila['wind_200mb']}","{fila['rh950mb']}","{fila['rh900mb']}",
                            "{fila["rh850mb"]}","{fila['rh800mb']}","{fila['rh750mb']}","{fila['rh700mb']}","{fila['rh650mb']}","{fila['rh600mb']}","{fila['rh550mb']}",
                            "{fila['rh500mb']}","{fila["rh450mb"]}","{fila['rh400mb']}","{fila['rh350mb']}","{fila['rh300mb']}","{fila['rh250mb']}","{fila['rh200mb']}", 
                            "{fila['timepoint']}","{fila['cloudcover']}","{fila["highcloud"]}","{fila['midcloud']}","{fila['lowcloud']}","{fila['temp2m']}","{fila['lifted_index']}",
                            "{fila['rh2m']}","{fila['msl_pressure']}","{fila['prec_amount']}","{fila['snow_depth']}","{fila['wind10m.speed']}","{fila['latitud_y']}",
                            "{fila['longitud_y']}","{id_paises}");
                            """
                print(query_clima)
                            
                self.crear_insertar_tabla(query_clima)
            # en vez de carga se pone el nombre de la variable que nosotros creemos cuando llamamos a 
            # la clase de crear tabla: carga.crear_insertar_tabla( query_medidas)
            
    
    

In [112]:
tabla_ataques= '''
CREATE TABLE IF NOT EXISTS `tiburones`.`ataques` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `id_paises` VARCHAR(100) NOT NULL,
  `year` VARCHAR (50) NOT NULL, 
  `age` VARCHAR (50) NOT NULL,
  `latitud_x` VARCHAR (50) NOT NULL,
  `longitud_x` VARCHAR (50) NOT NULL,
  `fatal_N` VARCHAR (50) NOT NULL,
  PRIMARY KEY (`id`))
ENGINE = InnoDB;
'''

tabla_clima= '''
CREATE TABLE IF NOT EXISTS `tiburones`.`clima` (
  `id_clima` INT NOT NULL AUTO_INCREMENT,
  `id` INT NOT NULL,
  `wind_950mb` VARCHAR (50) NOT NULL,
  `wind_900mb` VARCHAR (50) NOT NULL, 
  `wind_850mb` VARCHAR (50) NOT NULL, 
  `wind_800mb` VARCHAR (50) NOT NULL,
  `wind_750mb` VARCHAR (50) NOT NULL, 
  `wind_700mb` VARCHAR (50) NOT NULL, 
  `wind_650mb` VARCHAR (50) NOT NULL, 
  `wind_600mb` VARCHAR (50) NOT NULL, 
  `wind_550mb` VARCHAR (50) NOT NULL,
  `wind_500mb` VARCHAR (50) NOT NULL, 
  `wind_450mb` VARCHAR (50) NOT NULL, 
  `wind_400mb` VARCHAR (50) NOT NULL, 
  `wind_350mb` VARCHAR (50) NOT NULL, 
  `wind_300mb` VARCHAR (50) NOT NULL,
  `wind_250mb` VARCHAR (50) NOT NULL, 
  `wind_200mb` VARCHAR (50) NOT NULL, 
  `rh950mb` VARCHAR (50) NOT NULL, 
  `rh900mb` VARCHAR (50) NOT NULL, 
  `rh850mb` VARCHAR (50) NOT NULL, 
  `rh800mb` VARCHAR (50) NOT NULL,
  `rh750mb` VARCHAR (50) NOT NULL, 
  `rh700mb` VARCHAR (50) NOT NULL, 
  `rh650mb` VARCHAR (50) NOT NULL, 
  `rh600mb` VARCHAR (50) NOT NULL, 
  `rh550mb` VARCHAR (50) NOT NULL, 
  `rh500mb` VARCHAR (50) NOT NULL,
  `rh450mb` VARCHAR (50) NOT NULL, 
  `rh400mb` VARCHAR (50) NOT NULL, 
  `rh350mb` VARCHAR (50) NOT NULL, 
  `rh300mb` VARCHAR (50) NOT NULL, 
  `rh250mb` VARCHAR (50) NOT NULL, 
  `rh200mb` VARCHAR (50) NOT NULL,
  `timepoint` VARCHAR (50) NOT NULL, 
  `cloudcover` VARCHAR (50) NOT NULL,
  `highcloud`VARCHAR (50) NOT NULL,
  `midcloud` VARCHAR (50) NOT NULL,
  `lowcloud`VARCHAR (50) NOT NULL,
  `temp2m` VARCHAR (50) NOT NULL,
  `lifted_index` VARCHAR (50) NOT NULL,
  `rh2m` VARCHAR (50) NOT NULL,
  `msl_pressure` VARCHAR (50) NOT NULL,
  `prec_amount` VARCHAR (50) NOT NULL,
  `snow_depth` VARCHAR (50) NOT NULL,
  `wind10M.speed` VARCHAR (50) NOT NULL,
  `latitud_y` VARCHAR (50) NOT NULL,
  `longitud_y` VARCHAR (50) NOT NULL,
  PRIMARY KEY (`id_clima`),
  CONSTRAINT `fk_tabla_clima_tabla_ataques`
    FOREIGN KEY (`id`)
    REFERENCES `tiburones`.`ataques` (`id`))
ENGINE = InnoDB;
'''

In [113]:
bbdd="tiburones"

In [114]:
probando=Cargar_crear(bbdd)

In [115]:
crear_bbdd=probando.crear_bbdd()

Conexión realizada con éxito
MySQLCursor: CREATE DATABASE IF NOT EXISTS tiburones;


In [116]:
crear_tabla_ataque=probando.crear_insertar_tabla(tabla_ataques)

In [117]:
crear_tabla_clima=probando.crear_insertar_tabla(tabla_clima)

In [118]:
insertar_ataques=probando.insertar_datosi(df)

1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '27.527397260273972,"-24.7761,"134.755,"0.6902587519025876")' at line 2
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '27.527397260273972,"-24.7761,"134.755,"0.6902587519025876")' at line 2
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '27.174603174603174,"-41.5000831,"172.8344077,"0.7063492063492064")' at line 2
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '27.174603174603174,"-41.5000831,"172.8344077,"0.7063492063492064")' at line 2
1064 (42000): You have an error in your SQL syntax; ch